## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x<500].index.to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<1000].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
                                                       'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']])

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']

X = application_df[['STATUS', 'ASK_AMT']]
X = pd.concat([X, application_df_dummies], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimization #1

In [12]:
# make the function for hypertuning

def createModel(hp):

  # set up the sequential model
  nn = tf.keras.models.Sequential()

  # set up the choices for the activation functions needed in the hidden layers
  activationList = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

  # loop through a number of attributes on the first layer

  nn.add(
      tf.keras.layers.Dense(
          input_dim=len(X_train_scaled[0]),
          activation=activationList,
          units=hp.Int('first_units', min_value=1, max_value=80, step=5)
      ))

  # test different numbers of layers as well as neurons on those hidden layers
  for num in range(hp.Int('num_layers', 1, 11)):

    # add the layer (or layers) with a neuron (or neurons) on each layer
    nn.add(
        tf.keras.layers.Dense(
            activation=activationList,
            units=hp.Int(
                'units_' + str(num),
                min_value=1,
                max_value=80,
                step=5
            )
        )
    )

  # set up the output layer (1 node)
  nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  # compile the model
  nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  # return the model to used in the iteration
  return nn

In [13]:
pip install -q keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.1 MB/s eta 0:00:00


In [14]:
# import the keras-tuner modules
import keras_tuner as kt

In [15]:
# use kt Hyperband function to run the optimizer
tuner = kt.Hyperband(
    createModel,
    objective='val_accuracy',
    max_epochs=50,
    hyperband_iterations=2
)

In [16]:
# use the .search() function to find the best combo of hyperparameters
tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Trial 180 Complete [00h 02m 25s]
val_accuracy: 0.728863000869751

Best val_accuracy So Far: 0.7302623987197876
Total elapsed time: 01h 47m 27s


In [17]:
# get the best hyperparameters
tuner.get_best_hyperparameters(1)[0].values

{'activation': 'relu',
 'first_units': 31,
 'num_layers': 6,
 'units_0': 6,
 'units_1': 56,
 'units_2': 36,
 'units_3': 66,
 'units_4': 1,
 'units_5': 61,
 'units_6': 41,
 'units_7': 41,
 'units_8': 26,
 'units_9': 41,
 'units_10': 11,
 'tuner/epochs': 6,
 'tuner/initial_epoch': 2,
 'tuner/bracket': 3,
 'tuner/round': 1,
 'tuner/trial_id': '0020'}

In [18]:
# use get_best_models to evaluate the model loss and accuracy
bestDLModel = tuner.get_best_models(1)[0]

DLModel02Loss, DLModel02Accuracy = bestDLModel.evaluate(X_test_scaled, y_test)

268/268 [==============================] - 1s 3ms/step - loss: 0.5605 - accuracy: 0.7303


## Optimization #2

In [19]:
# import the random forest classifier module
from sklearn.ensemble import RandomForestClassifier

In [20]:
# create the random forest classifier model
rfModel = RandomForestClassifier(n_estimators=500, random_state=42)

In [21]:
# fit the model on the scaled training data
rfModel = rfModel.fit(X_train_scaled, y_train)

In [22]:
# make a list of predicted values using the scaled testing data
rfPredictedValues = rfModel.predict(X_test_scaled)
rfPredictedValues[:3]

array([0, 0, 0])

In [23]:
# import the modules for the accuracy score and classification report
from sklearn.metrics import accuracy_score, classification_report

# check the accuracy score
rfAccuracyScore = accuracy_score(y_test, rfPredictedValues)
print(f"Random Forest Accuracy Score: {rfAccuracyScore * 100:.2f}%")

Random Forest Accuracy Score: 70.83%


In [24]:
# extract the feauture importances from the random forest classifier
importantFeatures = rfModel.feature_importances_

# sort the features by their importance, zip them, then display them
zippedFeatures = sorted(zip(importantFeatures, X.columns), reverse=True)

zippedFeatures

[(0.3984788173196909, 'ASK_AMT'),
 (0.13516696172303838, 'AFFILIATION_CompanySponsored'),
 (0.12090951223020875, 'AFFILIATION_Independent'),
 (0.029037648096579896, 'APPLICATION_TYPE_T5'),
 (0.027290776352685488, 'ORGANIZATION_Association'),
 (0.023410794027700153, 'APPLICATION_TYPE_T10'),
 (0.021145228669769842, 'ORGANIZATION_Trust'),
 (0.020269469961019047, 'CLASSIFICATION_C2100'),
 (0.018496406523215995, 'CLASSIFICATION_Other'),
 (0.018182959157740607, 'APPLICATION_TYPE_T19'),
 (0.01782990854021599, 'APPLICATION_TYPE_T4'),
 (0.01688053462012358, 'CLASSIFICATION_C2000'),
 (0.01627734371860435, 'APPLICATION_TYPE_T3'),
 (0.013746078411878271, 'CLASSIFICATION_C1000'),
 (0.012519360529410295, 'USE_CASE_ProductDev'),
 (0.011393866819862724, 'APPLICATION_TYPE_T6'),
 (0.010909413699754856, 'USE_CASE_Preservation'),
 (0.010020794142804577, 'CLASSIFICATION_C1200'),
 (0.009401667575382817, 'INCOME_AMT_0'),
 (0.008197862168145545, 'CLASSIFICATION_C3000'),
 (0.00714338713285355, 'INCOME_AMT_2500

In [26]:
# Remove unimportant columns
X = X.drop(columns=['STATUS', 'USE_CASE_Other', 'AFFILIATION_Other'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_of_inputs = len(X_train_scaled[0])

hidden_nodes_layer_1 = 60
hidden_nodes_layer_2 = 40
hidden_nodes_layer_3 = 70
hidden_nodes_layer_4 = 60
hidden_nodes_layer_5 = 40
hidden_nodes_layer_6 = 40
hidden_nodes_layer_7 = 30
hidden_nodes_layer_8 = 40
hidden_nodes_layer_9 = 10


nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_1, activation="relu", input_dim=number_of_inputs))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_3, activation="relu"))

# Fourth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_4, activation="relu"))

# Fifth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_5, activation="relu"))

# Sixth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_6, activation="relu"))

# Seventh hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_7, activation="relu"))

# Eigth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_8, activation="relu"))

# Ninth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer_9, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 60)                2460      
                                                                 
 dense_12 (Dense)            (None, 40)                2440      
                                                                 
 dense_13 (Dense)            (None, 70)                2870      
                                                                 
 dense_14 (Dense)            (None, 60)                4260      
                                                                 
 dense_15 (Dense)            (None, 40)                2440      
                                                                 
 dense_16 (Dense)            (None, 40)                1640      
                                                                 
 dense_17 (Dense)            (None, 30)               

In [32]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the model
nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5734 - accuracy: 0.7153
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5582 - accuracy: 0.7261
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5531 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7300
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5524 - accuracy: 0.7302
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5507 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5498 - accuracy: 0.7314
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5497 - accuracy: 0.7322
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5484 - accuracy: 0.7319
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5489 - accura

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5564 - accuracy: 0.7315 - 1s/epoch - 4ms/step
Loss: 0.5563799738883972, Accuracy: 0.7315452098846436


## Optimization #3

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_of_inputs = len(X_train_scaled[0])

hidden_nodes_layer_1 = 80
hidden_nodes_layer_2 = 70
hidden_nodes_layer_3 = 70
hidden_nodes_layer_4 = 60
hidden_nodes_layer_5 = 50
hidden_nodes_layer_6 = 50
hidden_nodes_layer_7 = 40
hidden_nodes_layer_8 = 40
hidden_nodes_layer_9 = 30
hidden_nodes_layer_10 = 20


nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_1, activation="relu", input_dim=number_of_inputs))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_3, activation="relu"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_4, activation="relu"))

# Fifth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_5, activation="relu"))

# Sixth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_6, activation="relu"))

# Seventh hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_7, activation="relu"))

# Eigth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_8, activation="relu"))

# Ninth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_9, activation="relu"))

# Tenth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer_10, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 80)                3280      
                                                                 
 dense_22 (Dense)            (None, 70)                5670      
                                                                 
 dense_23 (Dense)            (None, 70)                4970      
                                                                 
 dense_24 (Dense)            (None, 60)                4260      
                                                                 
 dense_25 (Dense)            (None, 50)                3050      
                                                                 
 dense_26 (Dense)            (None, 50)                2550      
                                                                 
 dense_27 (Dense)            (None, 40)               

In [36]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
# Train the model
nn3.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 5s 3ms/step - loss: 0.5731 - accuracy: 0.7187
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5578 - accuracy: 0.7271
Epoch 3/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5546 - accuracy: 0.7301
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5538 - accuracy: 0.7293
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5525 - accuracy: 0.7301
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5507 - accuracy: 0.7307
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5506 - accuracy: 0.7318
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5505 - accuracy: 0.7320
Epoch 9/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5493 - accuracy: 0.7330
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5496 - accuracy: 0.7320

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5516 - accuracy: 0.7324 - 571ms/epoch - 2ms/step
Loss: 0.5516443848609924, Accuracy: 0.7323614954948425


In [42]:
# Export our model to HDF5 file
from tensorflow.keras.models import load_model, save_model
import warnings
warnings.simplefilter("ignore")
nn3.save('AlphabetSoupCharity_Optimization.h5')